# Autoencoders

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/m12sl/dl-hse-2020/blob/master/11-prob-models/autoencoders.ipynb)

Занимаемся автоэнкодерами.

<img src="https://github.com/m12sl/dl-hse-2020/raw/master/11-prob-models/img/encoder.png" crossorigin="anonymous"/>

План:
- написать несколько автоэнкодеров (AE, VAE) с разными размерностями внутреннних представлений.
- сделать реконструкцию нескольких примеров
- посмотреть на структуру внутреннего представления
- сделать интерполяцию между несколькими примерами

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import numpy as np
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict

from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets, transforms

# MNIST или FashionMNIST
download_path = '/tmp'
mnist_train = datasets.FashionMNIST(download_path, train=True, download=True, transform=transforms.ToTensor())
mnist_val = datasets.FashionMNIST(download_path, train=False, download=True, transform=transforms.ToTensor())

In [ ]:
plt.figure(figsize=[6, 6])
for i in range(4):
    plt.subplot(2, 2, i + 1)
    img, label = mnist_train[i]
    plt.title("Label: {}".format(label))
    plt.imshow(img[0, ...], cmap='gray')
    plt.axis('off')

Для тренировки и отображения графиков возьмем наш предыдущий код и несколько его улучшим для произвольного числа метрик:

1. Предлагается в коде модели написать метод `compute_all(batch)`, который будет вычислять предсказания, лосс и метрики
2. Возвращать из него словарь `dict(loss=loss, metrics=dict(name=value))`.


In [ ]:
def plot_history(log, name=None):
    """log is list of dictionaries like 
        [
            {'train_step': 0, 'train_loss': 10.0, 'train_acc': 0.0}, 
            ...
            {'train_step': 100, 'val_loss': 0.1, 'val_acc': 0.9},
            ...
        ]
    """
    if name is None:
        name='loss'
    train_points, val_points = [], []
    train_key = 'train_{}'.format(name)
    val_key = 'val_{}'.format(name)

    for entry in log:
        if train_key in entry:
            train_points.append((entry['train_step'], entry[train_key]))
        if val_key in entry:
            val_points.append((entry['train_step'], entry[val_key]))
    
    plt.figure()
    plt.title(name)
    x, y = list(zip(*train_points))
    plt.plot(x, y, label='train', zorder=1)
    x, y = list(zip(*val_points))
    plt.scatter(x, y, label='val', zorder=2, marker='+', s=180, c='orange')
    
    plt.legend(loc='best')
    plt.grid()
    plt.show()


def train_model(model, optimizer, train_dataset, val_dataset, batch_size=32, epochs=4):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    log = []
    train_step = 0
    for epoch in range(epochs):
        model.train()
        for x, _ in tqdm(train_loader):
            optimizer.zero_grad()
            # возможно этот фрагмент вы захотите переписать
            ret = model.compute_all(x)
            loss = ret['loss']
            #
            loss.backward()
            optimizer.step()
            train_metrics = {"train_" + key: value for key, value in ret['metrics'].items()}
            log.append({**train_metrics, 'train_step': train_step})
            train_step += 1

        tmp = defaultdict(list)
        model.eval()
        for x, _ in tqdm(val_loader):
            with torch.no_grad():
                # не забудьте и этот
                ret = model.compute_all(x)
                # 
                for key, value in ret['metrics'].items():
                    tmp[key].append(value)

        metrics = tmp.keys()
        val_metrics = {"val_" + key: np.mean(values) for key, values in tmp.items()}
        log.append({**val_metrics, 'train_step': train_step})
        
        clear_output()
        for m in metrics:
            plot_history(log, name=m)

## Обычный автоэнкодер

**Задание 1. (0.2):** Сделайте автоэнкодер с несколькими линейными слоями, разделенными RELU с внутренним представлением размером 2.

**Hint: для удобства добавьте методы compute_all/encode/decode**

In [ ]:
class AE(nn.Module):
    def __init__(self):
        super().__init__()
        # YOUR CODE HERE
        self.encoder = ...
        self.decoder = ...
        
    def compute_all(self, inp):
        <your code here>
        
        return dict(
            loss=loss,
            metrics=dict(
                # дополнительные метрики, какие хотите
                loss=loss.item(),
            )
        )
        

ae = AE()
opt = torch.optim.Adam(ae.parameters(), lr=1e-3)
train_model(ae, opt, mnist_train, mnist_val, epochs=3)

**Задание 1b (0.1):** Нарисуйте оригинал и восстановленную картинку

In [ ]:
def reconstruction_plot(model, dataset):
    plt.figure(figsize=[12, 3])
    N = 10
    idx = np.random.choice(len(dataset), size=N)
    for i in range(N):
        plt.subplot(2, N, i + 1)
        img, label = dataset[idx[i]]
        <your code here>
        rec = ...

        plt.title("id: {}".format(idx[i]))
        plt.imshow(img[0, ...], cmap='gray')
        plt.axis('off')

        plt.subplot(2, N, i + 1 + N)
        plt.title('label: {}'.format(label))
        plt.imshow(rec, cmap='gray')
        plt.axis('off')

    plt.subplots_adjust(wspace=0.03, hspace=0)
    
reconstruction_plot(ae, mnist_val)

**Задание 1c (0.1)**: отобразите внутренние представления натренерованного AE на плоскости

In [ ]:
# посмотрим на внутренние представления
def code_distribution(model, dataset):
    N = 1000
    idx = np.random.choice(len(dataset), size=N)
    
    <your code here>
    x, y, colors = ...
    
    # если в x, y, colors лежат координаты и метки, то следующий код выведет график с легендой
    cm=plt.cm.rainbow(np.linspace(0,1,10))
    plt.figure(figsize=(12, 12))
    plt.scatter(x, y, c=cm[np.array(colors)])
    plt.grid()
    plt.legend(handles=[mpatches.Patch(color=cm[i], label='{}'.format(i)) for i in range(10)])
    plt.show()
    
code_distribution(ae, mnist_train)

**Какие особенности вы видите на этой картинке?**

**Задание 1d (0.1)** Интерполяция! 
1. Выберете случайным образом несколько пар изображений, преобразуйте их в _коды_.
2. Постройте линейную интерполяцию точек между ними (всего 10 точек, крайние -- это выбранные пары).
3. Декодируйте коды в картинки.


In [ ]:
def plot_interpolations(model, dataset):
    N = 10
    idx = np.random.choice(len(dataset), size=N)
    
    # your code here
    out = ...
    # end of your code
    # если out имеет размер [5, 10, 28, 28], следующий код нарисует таблицу с интерполяционными картинками
    
    fig, axes = plt.subplots(nrows=5, ncols=10, figsize=(14, 7),
                             subplot_kw={'xticks': [], 'yticks': []})
    
    for i in range(50):
        axes[i // 10, i % 10].imshow(out[i // 10, i % 10], cmap='gray')
    

plot_interpolations(ae, mnist_val)

**Какие особенности вы видите на этих картинках?**

## Вариационный автоэнкодер (не Байесовский вывод)


Предположим, мы хотим, чтобы `code` был распределен нормально (стандартное 2d распределение).
Будем 


1. Как сделать одно распредление похожим на другое? Минимизировать KL-дивергеницю!
 
 $KL(q(x)||p(x)) = \int q(x) \log \frac{q(x)}{p(x)}$
 
2. Как сделать из `code` распределение, а не одну точку? Давайте сделаем из него распределение!
 
 $z = \mu + \varepsilon \sigma$ где $code = [\mu, \sigma]$, а $\varepsilon \sim N(0, 1)$
 
3. Мы можем не прописывать все распределение. Между модельными распредлениями дивергенцию можно прописать аналитически.

 $z \sim N(\mu, \sigma)$
 
 $KL = \sum\limits_{i} \mu^2 + \sigma^2 - \log \sigma - 1$
 
 
<img src="https://github.com/m12sl/dl-hse-2020/raw/master/11-prob-models/img/vae.png" crossorigin="anonymous"/>
 
 




## Что можно посмотреть про байесовский вывод?
- [Лекция Ветрова DeepBayes2017. Модели с латентными переменными](https://youtu.be/7yLOF07Mv5I)
- [Лекция Ветрова DeepBayes2017. Масштабируемые байесовские методы](https://youtu.be/if9bTlZOiO8) << тут про VAE
- [Auto-Encoding Variational Bayes](https://arxiv.org/pdf/1312.6114.pdf)
- [Tutorial on Variational Autoencoders](https://arxiv.org/pdf/1606.05908.pdf)
- [blogpost](https://towardsdatascience.com/intuitively-understanding-variational-autoencoders-1bfe67eb5daf)
- [KL дивергенция между двумя нормальными распределениями](https://stats.stackexchange.com/questions/7440/kl-divergence-between-two-univariate-gaussians)



**Задание 2 (0.3)** Сделайте вариационный автоэнкодер с несколькими линейными слоями, разделенными RELU с внутренним представлением размером 2.

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = ...
        self.decoder = ...
        
    def compute_all(self, inp):
        <your code here>
        return dict(
            loss=loss,
            metrics=dict(
                # любые метрики на ваш вкус
                kld=kld.item(),
                loss=loss.item(),
            )
        )
vae = VAE()
opt = torch.optim.Adam(vae.parameters(), lr=1e-3)
train_model(vae, opt, mnist_train, mnist_val, epochs=3)

**Задание 2b (0.2)** 
- Постройте реконструкционные примеры
- Постройте распределение внутренних представлений
- Постройте табличку с интерполяциями

**Изменилось ли что-то по сравнению с обычным автоэнкодером?**

In [ ]:
reconstruction_plot(vae, mnist_val)

In [ ]:
code_distribution(vae, mnist_train)

In [ ]:
plot_interpolations(ae, mnist_val)

**Задание 3 (0.5+)** Увеличьте качество реконструкции (mae/mse-loss/тюнинг параметров или тренировка на пониженном LR, можно увеличить внутреннее представление). Постройте табличку с реконструкцией и интерполяцией.

Полезные применения (*)автоэнкодеров:
 - претрейн на неразмеченных данных
 - уменьшение размерности
 - полезные для других моделей представления
 - трактуемые внутренние представления
 - детектирование аномалий
 - denoising
 - манипуляции внутренними представлениями
 - синтез новых примеров
 - ...


Для решения некоторых из этих задач не обязательно завязываться на encoder-decoder-архитектуру, посмотрите на i-RevNets (Invertible Reversible Networks):


 